# MDP and gym

## Evaluation considerations
- We take into account the correctness of the solutions but also their generality and quality of the code
- Comment and discuss on the results of all your exercises (in a cell immediately after the results). You may also state the difficulties encountered, lessons learned and your understanding of the problem and solution
- Clean-up your code before submission, do not leave unnecessary code attempts, or if you deem it important, leave them in a way that it is easily understood and with comments/discussion
- We also value the originality of the solutions, don't hesitate in performing unrequested additional tasks in relation to the exercises


**NOTE**

Do not try to reproduce exactly the results in this notebook. RL training is very noisy and performances of learned policies can vary a lot, try running your trains several times.



In [ ]:
!pip install gymnasium
!pip install gymnasium[accept-rom-license,toy_text]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 KB 14.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 66.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached libtorrent-2.0.7-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (8.6 MB)


In [ ]:
import gymnasium as gym
from IPython.display import clear_output, HTML, display
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
#@title Wrapper for recording an environment into a video

from __future__ import annotations

from copy import deepcopy
from typing import Any, SupportsFloat

from gymnasium.core import ActType, ObsType, RenderFrame, WrapperActType, WrapperObsType
from gymnasium.error import DependencyNotInstalled

class RecordVideo(gym.Wrapper):
    """Adapted from https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/experimental/wrappers/rendering.py#L87
    """

    def __init__(self, env):
        """Initialize a :class:`HumanRendering` instance.
        Args:
            env: The environment that is being wrapped
        """
        super().__init__(env)
        assert env.render_mode in [
            "rgb_array",
            "rgb_array_list",
        ], f"Expected env.render_mode to be one of 'rgb_array' or 'rgb_array_list' but got '{env.render_mode}'"

        if "render_fps" not in env.metadata:
            env.metadata["render_fps"] = 24

        assert (
            "render_fps" in env.metadata
        ), "The base environment must specify 'render_fps' to be used with the HumanRendering wrapper"

        if "human" not in self.metadata["render_modes"]:
            self.metadata = deepcopy(self.env.metadata)
            self.metadata["render_modes"].append("human")

        self.artists = []
        self.figure = None
        self.env = env
    @property
    def render_mode(self):
        """Always returns ``'human'``."""
        return "human"

    def step(
        self, action: WrapperActType
    ) -> tuple[WrapperObsType, SupportsFloat, bool, bool, dict]:
        """Perform a step in the base environment and render a frame to the screen."""
        result = super().step(action)
        self._render_frame()
        return result

    def reset(
        self, *, seed: int | None = None, options: dict[str, Any] | None = None
    ) -> tuple[WrapperObsType, dict[str, Any]]:
        """Reset the base environment and render a frame to the screen."""
        result = super().reset(seed=seed, options=options)
        self._render_frame()
        return result

    def video(self):
        """This method renders all frames collected up to now."""
        if self.figure is not None:
            from IPython.display import HTML
            import matplotlib.animation

            animation = matplotlib.animation.ArtistAnimation(self.figure, self.artists, 
                                                             interval=1000//self.metadata["render_fps"],
                                                             blit=True,
                                                             repeat=True,
                                                             repeat_delay=2000)
            return HTML(animation.to_html5_video())

        return None

    def _render_frame(self):
        """Fetch the last frame from the base environment and render it to the screen."""
        try:
            import matplotlib.animation
            import numpy as np
        except ImportError:
            raise DependencyNotInstalled(
                "matplotlib is not installed, run `pip install matplotlib`"
            )
        if self.env.render_mode == "rgb_array_list":
            rgb_arrays = self.env.render()
        elif self.env.render_mode == "rgb_array":
            rgb_arrays = [self.env.render()]
        else:
            raise Exception(
                f"Wrapped environment must have mode 'rgb_array' or 'rgb_array_list', actual render mode: {self.env.render_mode}"
            )

        assert isinstance(rgb_arrays, list)

        for rgb_array in rgb_arrays:
            assert isinstance(rgb_array, np.ndarray)

        if self.figure is None:
            self.figure = plt.figure()
            plt.axis('off')
        
        self.artists.append([plt.imshow(rgb_array) for rgb_array in rgb_arrays])

    def close(self):
        """Close the rendering window."""
        result = self.video()
        super().close()

        return result

In [ ]:
import numpy as np
np.set_printoptions(linewidth=100)

Let's render soe steps to show how to use the `RecordVideo` class.

In [ ]:
env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
env.reset()                    

for _ in range(100):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

    done = terminated or truncated

    if done:
        env.reset()

display(env.video())

This is only needed when we want to render. In many situations of this lab we will perform many steps without the need to render, we will avoid recording a video, since doing so requires a lot of RAM (to save all the RGB frames) and we may run out of memory.

To avoid this do not wrap the environment on a `RecordVideo` class and set keyword argument `render_mode=None` when making the environment.

In [ ]:
env = gym.make("FrozenLake-v1", render_mode=None)
env.reset()                    

for _ in range(100):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

    done = terminated or truncated

    if done:
        env.reset()

In this lab we will be estimating value functions. Value functions of an MDP are conditioned on the policy used.

A policy is the probability distribution over actions. We will restrict ourselves to discrete action spaces.

We will start by using a uniform policy that chooses with equal chances between all actions.

## Exercise : A uniform policy

Create a `policy_uniform` function that returns the probability of each possible action given an environment and state.

We will also create a `sample_multinomial` function that facilitates sampling an action from the policy given the probability of each possible action.

**Hint** you may access the number of discrete actions with `env.action_space.n`.

In [ ]:
class UniformPolicy(object):
    def __init__(self, action_space):
        assert isinstance(action_space, gym.spaces.discrete.Discrete), "Can only create uniform policies for Discrete action spaces"
        # print(action_space)
        self.n_actions = action_space.n
        self.training = True

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    def probability(self, state, action):
        ### BEGIN SOLUTION
        action_prob = 1/self.n_actions
        ### END SOLUTION

        return action_prob

    def sample(self, state):
        ### BEGIN SOLUTION
        choices = [i for i in range(self.n_actions)]
        # print(state)
        action = np.random.choice(choices)
        # action = np.random.randint(self.n_actions)
        ### END SOLUTION
        
        return action

# Let's instantiate a uniform policy
env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
uniform_policy = UniformPolicy(env.action_space)

# And sample 20 actions from state==0
actions = [uniform_policy.sample(0) for i in range(20)]
print(actions)

Gym follows a formalism of MDPs that differs slightly (is more general) than the one we have seen in class. The differences are:
- In gym, transitions from a particular state and action can lead to the same destination state with different rewards (in the course all transitions reaching the same state obtain the same reward, except for transitions departing from a terminal state, whose rewards can be ignored)
- In gym, transitions to the same state can be both terminal and non-terminal (in the course it's the states that are terminal)

However we have chosen the environment `FrozenLake` which behaves as the MDPs described in the course, thus:
- All transitions to a given state have the same reward (unless the departure and destination state is the same terminal state)
- All transitions to a terminal state are terminal

In bym `env.env.P` contains the information about the process dynamics, rewards and terminal states. The information is encoded in the following way:

`env.env.P[state][action]` returns a list of tuples containing all the possible outcomes and their probability in the form `(prob, next_state, reward, terminal)` where `state` is the current state, `action` is the action taken, `probability` is the probability of the transition, `next_state` is the destination state of the transition, `reward` is the reward obtained for the transition and `terminal` is a boolean indicating if the transition ends the episode.

## Exercise : MDP from gym environment

Using this information compute the matrices $\mathcal{P}_{ss'}$, $\mathcal{R}_s$ and `terminal_states` of your MDP. Terminal is a boolean vector containing `True` for terminal states and `False` otherwise.

We will use the uniform policy that we created before, that assigns equal probability to all actions.

**Hint** all the transitions from a terminal state must be ignored.

**Optionally** while computing the matrices verify with `assert` that the conditions that we described above are fulfilled.

In [ ]:
def compute_P_and_R(env, policy):
    # Initialize the matrices P and R
    n_states = env.observation_space.n
    P = np.zeros((n_states, n_states))

    R = np.empty((n_states, 1))
    R[:] = np.nan

    # Initialize a vector for terminal states
    # we will initialize with nan
    terminal_states = np.empty(n_states)
    terminal_states[:] = np.nan

    ### BEGIN SOLUTION
    # Iterate over env.env.P.items()
    # env.env.P is a dictionary that maps from 
    # departure state to a dictionary of possible actions and subsequent transitions


        # Iterate over all possible actions with equal probability


            # Obtain the probability of taking that action


            # Iterate over all transitions for this action

                # If the next state is terminal flag it as such in terminal_states



                # If we are not departing from a terminal state

                    # Add to P the probability of the transition as:
                    # (probability of the action) * (probability of the transition)


                    # Set the reward in R



    ### END SOLUTION
    N = env.observation_space.n
    P = np.zeros((N,N))
    num_of_action = env.action_space.n
    terminal_states = np.empty(N)
    terminal_states[:] = np.nan
    for i in range(N):
        state = env.env.P[i]
        prob_to_take_a_action = 1/num_of_action
        for action in state:
            state_action_dynamics = state[action]
            # print(f'state:{i}, action: {action}')
            # print(state_action_dynamics)
            for result in state_action_dynamics:
                
                next_state = result[1]
                prob = result[0]
                is_terminal = result[3]
                terminal_states[next_state] = is_terminal
                # if P[i,next_state]!=0:
                #     print('found_something')
                #     print(prob)
                #     print(P[i,next_state])
                    
                P[i,next_state] += prob*prob_to_take_a_action
        
            

    # Convert the terminal states vector to boolean to use in indexing
    terminal_states = terminal_states.astype(bool)
    R = np.zeros((N,1))
    R[N-1,0] = 1

    return P, R, terminal_states

env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
P, R, terminal_states = compute_P_and_R(env, uniform_policy)

print(P)
print(R)
print(terminal_states)

## Exercise : Sample an episode

Sample an episode with the uniform policy and return the lists of observations (states), actions, rewards and the done flags.

Notice that the list of states should have one more element (the initial state).

Test the function by rendering an episode. We will use `RecordVideo` to do that.

In [ ]:
env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery = True))
env.reset()
count = 0
for i in range(100):
    action = env.action_space.sample()
    
    obs, reward, terminated, truncated, info = env.step(action)
    # print(obs)
    done = terminated or truncated
    
    if done:
        count+=1
        env.reset()

print(count)
display(env.video())

In [ ]:
def sample_episode(env, policy, reset=True):
    states = []
    actions = []
    rewards = []
    dones = []

    ### BEGIN SOLUTION
    # print('i am here')
    # If reset, we reset the environment and get an initial state
    # else we set the initial state to it's current state env.env.s
    if reset:
        # print('i am here hehe')
        initial_state,_ = env.reset()
        # print('done')
        # print(initial_state)
    else:
        initial_state = env.env.s
        # print('i am also here hehe')

    done = False
    # Collect the initial state
    states.append(initial_state)
    
    # While the episode has not finished
    while not done:
        
        # Select an action
        action = policy.sample(states[-1])
        actions.append(action)
        
        # Step the environment
        obs, reward, terminated, truncated, info = env.step(action)
        
        # The episode is done if it has been terminated or truncated
        done = terminated or truncated 

        # print(obs)
        # Collect the state, reward and action taken
        states.append(obs)
        rewards.append(reward)
        dones.append(done)



    ### END SOLUTION

    return states, actions, rewards, dones

env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
uniform_policy = UniformPolicy(env.action_space)
states, actions, rewards, dones = sample_episode(env, uniform_policy)
print(rewards)
print(len(states), len(actions), len(rewards), len(dones))
# print(states)
display(env.video())

## Exercise : Returns of episode

For a sampled episode compute the return $G_t$ for all steps $t$.

**Hint** the return is easily computed backwards from the last step in the episode to the first.

In [ ]:
def compute_returns(rewards, gamma):
    returns = np.zeros(len(rewards))

    ### BEGIN SOLUTION
    #changed this
    # rewards = np.array(rewards)/len(rewards)
    # Iterate over the rewards backward computing each return
    
    # using the previous return computed
    episode_len = len(rewards)
    # print(f"before return {returns}, {episode_len}, {rewards[episode_len -1]}")
    returns[episode_len-1] = rewards[episode_len-1]
    # print(f"first return {returns}")
    
    for i,reward in enumerate(reversed(rewards)):
        if i==0:
            # print(f"{episode_len-1-i} and {rewards[episode_len-1-i]} and {gamma*rewards[episode_len-i-1]}, {returns[episode_len-1-i]}") 
            continue

        returns[episode_len-1-i] = rewards[episode_len-1-i] + gamma*returns[episode_len-i]
        # print(f"{episode_len-1-i} and {rewards[episode_len-1-i]} and {gamma*returns[episode_len-i]}, {returns[episode_len-1-i]}")
        
        


    ### END SOLUTION

    return returns

env = gym.make("FrozenLake-v1", render_mode="rgb_array")
uniform_policy = UniformPolicy(env.action_space)

# Now we will run until one of them has a positive reward
while True:
    states, actions, rewards, dones = sample_episode(env, uniform_policy)

    if np.sum(rewards) > 0:
        # Print the rewards
        print(rewards)

        # Compute and print the returns
        gamma = 0.9
        returns = compute_returns(rewards, gamma)
        print(returns)

        # Exit the loop
        break

# Policy evaluation

In this section we will implement various value function estimation methods:
- Direct method using the previous P and R
- Naive (cheating) method that assumes possibility of setting an arbitrary initial state
- Monte Carlo methods
- Time-difference method

## Exercise : Direct method

Solve the Bellman equation, using the direct solution (matrix inversion).

**Note** that this method leads to miss-leading values for terminal states, since there is no way to indicate terminality using the Bellman equation without sampling.

In [ ]:
def value_direct(env, gamma, policy):
    P, R, terminal_states = compute_P_and_R(env, policy)
    P[env.observation_space.n-1, env.observation_space.n-1] = 0 
    ### BEGIN SOLUTION
    I = np.eye(env.observation_space.n)
    v = np.linalg.inv(I-gamma*P)@R   

    ### END SOLUTION

    # Unsqueeze the extra dimension
    v = v[:,0]

    return v
    
env = gym.make("FrozenLake-v1", render_mode="rgb_array")
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
v_direct = value_direct(env, gamma, uniform_policy)
print(v_direct)

## (Optional) Exercise : Verify the solution

Check if the computed value function $V$ fulfills the Bellman equation. You may use `np.allclose(a, b)` to check if all elements in `a` and `b` are close up to a numerical error.

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

For the following we must set the value function of terminal states to 0 in order to keep our value function comparable to those computed by methods using episode sampling.

In [ ]:
# To compare to the following methods we must set the value of terminal states to 0
v_direct[terminal_states] = 0
print(v_direct)

## Exercise : Naive (cheating) method

Compute the value function for each state by cheating (changing the starting state and computing the average return from the start).

In [ ]:
def value_naive(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION

    # Initialize values and counts tables (one cell per state)
    values = np.zeros((env.observation_space.n,1))
    counts = np.zeros((env.observation_space.n,1))


    # Compute the number of episodes per state
    episodes_per_state = int(n_episodes/env.observation_space.n)


    # For each initial state
    for i in range(env.observation_space.n):

        # For each episode
        for episode in range(episodes_per_state):
            # Reset the environment
            env.reset()

            # Set the initial state
            env.env.s = i

            # Sample an episode 
            # (without reseting the environment to avoid changing the initial state)
            states, actions, rewards, dones = sample_episode(env, policy, reset = False)

            # Compute the returns
            # gamma = 0.9
            returns = compute_returns(rewards, gamma)
            
            # Accumulate the return of the initial state
            values[i] += returns[0]

        # Divide the accumulated value by the number of episodes
        values[i] = values[i]/episodes_per_state

    ### END SOLUTION
    
    return values

def compute_value_error(v_est, v_ref):
    diff = (v_est - v_ref)
    return np.mean(diff), np.std(diff)


env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_naive = value_naive(env, gamma, uniform_policy, n_episodes)
print(v_naive)
print(compute_value_error(v_naive, v_direct))

## Exercise : Monte Carlo first visit method
Compute the value function for each state using the Monte carlo method "first visit"

In [ ]:
def value_montecarlo_first(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION

    # Initialize values and counts tables (one cell per state)
    values = np.zeros((env.observation_space.n,1))
    counts = np.zeros((env.observation_space.n,1))

    # For each episode
    for episode in range(n_episodes):
        # Sample an episode and compute returns
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        # print(states)
        # Keep track of visited states
        visited_states = set(states)
        returns = compute_returns(rewards, gamma)

        # For each state and associated return
        for state, ret in zip(states, returns):
            # If first visit
            if state in visited_states:
                
                # Increment counts
                counts[state]+=1

                # Accumulate returns
                values[state]+=ret 

                # Update the set of visited states
                visited_states.remove(state)

    # Average the accumulated returns
    for i in range(env.observation_space.n):
        if counts[i]!=0:
            values[i] = values[i]/counts[i]

    ### END SOLUTION
    
    return values

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_mc_firstvisit = value_montecarlo_first(env, gamma, uniform_policy, n_episodes)
print(v_mc_firstvisit)
print(compute_value_error(v_mc_firstvisit, v_direct))

## Exercise : Monte Carlo every visit method
Compute the value function for each state using the Monte carlo method "every visit"

In [ ]:
def value_montecarlo_every(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION
    # Initialize values and counts tables (one cell per state)
    values = np.zeros((env.observation_space.n,1))
    counts = np.zeros((env.observation_space.n,1))

    # For each episode
    for episode in range(n_episodes):
        # Sample an episode and compute returns
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        returns = compute_returns(rewards, gamma)

        # For each state and associated return
        for state, ret in zip(states, returns):
            # Increment counts
            counts[state]+=1

            # Accumulate returns
            values[state]+=ret 

    # Average the accumulated returns
    for i in range(env.observation_space.n):
        if counts[i]!=0:
            values[i] = values[i]/counts[i]


    ### END SOLUTION
    
    return values

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_mc_everyvisit = value_montecarlo_every(env, gamma, uniform_policy, n_episodes)
print(v_mc_everyvisit)
print(compute_value_error(v_mc_everyvisit, v_direct))

## Exercise : Monte Carlo incremental

Implement the incremental Monte Carlo method.

In [ ]:
def value_montecarlo_incremental(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION
    # Initialize values and counts tables (one cell per state)

    values = np.zeros((env.observation_space.n,1))
    counts = np.zeros((env.observation_space.n,1))



    # For each episode
    for episode in range(n_episodes):
    
        # Sample an episode and compute returns
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        returns = compute_returns(rewards, gamma)

        # For each state and associated return
        for state, ret in zip(states, returns):
        
            # Increment counts
            counts[state]+=1

            # Update value with return
            values[state] = values[state]+ (ret-values[state])/counts[state]

    ### END SOLUTION
    return values

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_mc_incremental = value_montecarlo_incremental(env, gamma, uniform_policy, n_episodes)
print(v_mc_incremental)
print(compute_value_error(v_mc_incremental, v_direct))

## Exercise : Time-differences (TD) method

In [ ]:
import gymnasium as gym
def value_td(env, gamma, policy, n_episodes, alpha=0.4):
    ### BEGIN SOLUTION
    # Initialize value table (one cell per state)
    values = np.zeros((env.observation_space.n,1))


    # For each episode
    for episode in range(n_episodes):
        # Sample an episode
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        returns = compute_returns(rewards, gamma)
        # print(len(states))
        # print(len(rewards))
        # For each step in the episode
        for i,state in enumerate(states):
            # Update the value of the depart state with the current value, the value of the next state and the reward
            # print(i)
            if i<len(rewards):
                values[state] = values[state] + alpha*(rewards[i] + gamma*values[states[i+1]] - values[state])
        
        ### END SOLUTION
    return values


env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
alpha = 0.4
v_td = value_td(env, gamma, uniform_policy, n_episodes, alpha=alpha)
print(v_td)
print(compute_value_error(v_td, v_direct))

## (Optional) Exercise : Comparison TD to Monte Carlo incremental

How do these to methods compare? Explain the relative advantages and disadvantages.

## Exercise : Action-value
Compute the action value function using one or more of the following methods:
- Naive (cheating)
- MC "first visit"
- MC "every visit"
- MC incremental
- TD

In [ ]:
def actionvalue_montecarlo_every(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION
    q = np.zeros((env.observation_space.n,  env.action_space.n))
    counts = np.zeros((env.observation_space.n,  env.action_space.n))

    for episode in range(n_episodes):
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        returns = compute_returns(rewards, gamma)

        for state, action, ret in zip(states, actions, returns):
            # Increment counts
            counts[state,action]+=1

            # Accumulate returns
            q[state, action]+=ret 

    # Average the accumulated returns
    for i in range(env.observation_space.n):
        for j in range(env.action_space.n):
            if counts[i,j]!=0:
                q[i,j] = q[i,j]/counts[i,j]
                # values[i] = values[i]/counts[i]


    ### END SOLUTION
    return q

env = gym.make("FrozenLake-v1", render_mode="rgb_array")
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000*5
q_mc_everyvisit = actionvalue_montecarlo_every(env, gamma, uniform_policy, n_episodes)
print(q_mc_everyvisit)

# Policy improvement

In this section we are going to improve upon the uniform policy, which selects actions at random, independently on the state.

To assess whether our learned policies work, we will start by implmenting a scoring function to evaluate the policies.

## Exercise : Evaluate performance of a policy

In this environment we define a goal as obtaining a final reward greater than 0, thus reaching the target state, since it is the only one with a non-zero reward.

For a given policy, compute:
- the average number of episodes where the goal was reached
- the average number of steps to reach the goal


In [ ]:
def score_policy(env, gamma, policy, n_episodes):
    episodes_to_goal = 0
    steps_to_goal = []

    ### BEGIN SOLUTION
    for episode in range(n_episodes):
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        if rewards[-1]!=0:
            episodes_to_goal+=1
            steps_to_goal.append(len(rewards))




    ### END SOLUTION

    return episodes_to_goal/n_episodes, np.mean(steps_to_goal)

score_policy(env, 0.9, uniform_policy, 10000)

## Exercise : Greedy policy (policy improvement)

Create a greedy policy from an action value function and evaluate it's performance.

The greedy policy selects the action leading to the largest value in the current state.

**Note** that when several actions have the same maximal value, it is best to randomly pick one of them.

Pick the first highest value action or (optionally) pick randomly amongst actions with the highest value.

In [ ]:
class GreedyPolicy(UniformPolicy):
    def __init__(self, action_space, q):
        super().__init__(action_space)

        # q is the action-value table
        self.q = q

    def _max_value_action(self, state):
        ### BEGIN SOLUTION

        action = np.argmax(self.q[state])
        ### END SOLUTION
        return action

    def probability(self, state, action):
        # Select the highest value action
        action_max = self._max_value_action(state)

        # Return a probability of 1 for the selected action 0 otherwise
        action_prob = float(action == action_max)
        
        return action_prob

    def sample(self, state):
        # Select the highest value action
        action_max = self._max_value_action(state)

        return action_max

env = gym.make("FrozenLake-v1", render_mode=None)
greedy_policy = GreedyPolicy(env.action_space, q_mc_everyvisit)

## Exercise : Compare the performance of policies

Report the performance of the uniform policy and the greedy policy

In [ ]:
# env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery = True))
env = gym.make("FrozenLake-v1", render_mode=None)

### BEGIN SOLUTION

un_per,un_mean =  score_policy(env, 0.9, uniform_policy, 10000)
print(f"uniform policy: {un_per:.2%} / {un_mean}")

gr_per,gr_mean = score_policy(env, 0.9, greedy_policy, 10000)
print(f"greedy policy: {gr_per:.2%} / {gr_mean}")


### END SOLUTION

In [ ]:
# env.render_mode = "rgb_array"
env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery = True))
for i in range(10):
    states, actions, rewards, dones = sample_episode(env, greedy_policy)
display(env.video())

# Policy learning

In this section we will start learning policies.

We will then implement the following policy learning methods:
- policy iteration
- SARSA
- Q-Learning

## Exercise : Policy iteration
By alternating between policy evaluation and policy improvement, find an optimal policy.

Print the scores of each intermediate policy and comment on how the metrics evolve.

In [ ]:
# Initial policy: 1.80% / 14.28
# Policy (iter: 0): 2.80% / 12.54, Value change: nan
# Policy (iter: 1): 3.20% / 12.00, Value change: 2.97
# Policy (iter: 2): 2.70% / 11.52, Value change: 2.46
# Policy (iter: 3): 3.40% / 12.38, Value change: 2.12
# Policy (iter: 4): 6.00% / 15.42, Value change: 4.32
# Policy (iter: 5): 6.60% / 14.41, Value change: 4.05
# Policy (iter: 6): 7.50% / 15.71, Value change: 0.36
# Policy (iter: 7): 7.00% / 16.91, Value change: 0.84
# Policy (iter: 8): 7.00% / 13.84, Value change: 0.88
# Policy (iter: 9): 7.70% / 16.21, Value change: 0.45
# Policy (iter: 10): 6.00% / 15.52, Value change: 0.44
# Policy (iter: 11): 6.20% / 15.35, Value change: 0.62
# Policy (iter: 12): 7.10% / 15.46, Value change: 0.98
# Policy (iter: 13): 8.10% / 15.41, Value change: 0.74
# Policy (iter: 14): 8.10% / 16.12, Value change: 0.43
# Policy (iter: 15): 9.00% / 15.02, Value change: 0.60
# Policy (iter: 16): 7.60% / 16.47, Value change: 0.60
# Policy (iter: 17): 7.20% / 16.10, Value change: 0.58
# Policy (iter: 18): 6.80% / 16.54, Value change: 0.52
# Policy (iter: 19): 8.80% / 17.15, Value change: 0.88
# Policy (iter: 20): 7.20% / 16.44, Value change: 0.45
# Policy (iter: 21): 5.60% / 11.82, Value change: 0.48
# Policy (iter: 22): 6.80% / 12.63, Value change: 1.43
# Policy (iter: 23): 5.00% / 11.94, Value change: 1.30
# Policy (iter: 24): 3.80% / 14.11, Value change: 0.51
# Policy (iter: 25): 7.90% / 15.90, Value change: 2.13
# Policy (iter: 26): 11.80% / 16.68, Value change: 2.09
# Policy (iter: 27): 9.50% / 17.68, Value change: 0.82
# Policy (iter: 28): 8.90% / 16.01, Value change: 0.27
# Policy (iter: 29): 9.30% / 15.66, Value change: 1.02

In [ ]:
def policy_learn_iteration(env, initial_policy, policy_evaluation_function, 
                           n_episodes_value, n_episodes_score, n_iterations):
    policy = initial_policy

    ### BEGIN SOLUTION

    # Score and print the initial policy
    per,mean = score_policy(env, 0.9, policy, n_episodes_score)
    print(f"Initial Policy: {per:.2%} / {mean}")
    action_value_prev = None
        # Policy evaluation
    for it in range(n_iterations):
        if it==0:
            per,mean = score_policy(env, 0.9, policy, n_episodes_score)
            print(f"Policy (iter: {it}): {per:.2%} / {mean:.4}, Value change: nan")
            

        # Keep track of the action-value function change 
        # (sum of absolute difference between the previous 
        #  and next action-value funcitons)
        # Note that at the first step we will report a change of nan
        # since we still don't have a previous action-value function
        action_value = policy_evaluation_function(env, gamma, policy, n_episodes_value)
        if it!=0:
            diff = np.abs(np.diff(action_value_prev - action_value))
            
        action_value_prev = action_value
        
        # Policy improvement
        policy = GreedyPolicy(env.action_space, action_value)

        # Policy scoring
        if it!=0:
            
            per,mean = score_policy(env, 0.9, policy, n_episodes_score)
            print(f"Policy (iter: {it}): {per:.2%} / {mean:.4}, Value change: {np.sum(diff):.3}")
            
        



    ### END SOLUTION

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)

policy_learn_iteration(env, uniform_policy, actionvalue_montecarlo_every, 
                       n_episodes_value=1000, n_episodes_score=10000,
                       n_iterations=100)

## Exercise : Epsilon-greedy policy

One of the issues with the greedy policy is that some possible trajectories may never be visited depending on the initial estimation of the value function.

To avoid this create an epsilon-greedy policy. These policies act differently when running in training mode and evaluation mode. In evaluation mode they act as a greedy policy. In training mode:
- with probability epsilon, uniformly selects an action
- else selects the action with maximum value (greedy policy)

Implement the `EpsilonGreedyPolicy` class.


In [ ]:
class EpsilonGreedyPolicy(GreedyPolicy):
    def __init__(self, action_space, q, epsilon, 
                 epsilon_decay=1, epsilon_min=0):
        super().__init__(action_space, q)
        self.epsilon_start = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        self.epsilon = self.epsilon_start

    def sample(self, state):
        ### BEGIN SOLUTION
        if np.random.random()<= self.epsilon and self.training:
            choices = [i for i in range(self.n_actions)]
            action = np.random.choice(choices)
        else:
            action = np.argmax(self.q[state])



        ### END SOLUTION
        return action

    def begin_episode(self, episode_index):
        # Start of an episode
        self.epsilon = self.epsilon_start * (self.epsilon_decay ** episode_index)
        self.epsilon = max(self.epsilon, self.epsilon_min)

# Instantiate a policy
dummy = EpsilonGreedyPolicy(env.action_space, q_mc_everyvisit, epsilon=0.5)

# Sample 20 actions from state 0 in train mode
dummy.train()
actions = [dummy.sample(2) for i in range(20)]
print(actions)

# Sample 20 actions from state 0 in eval mode
dummy.eval()
actions = [dummy.sample(2) for i in range(20)]
print(actions)

## Exercise : SARSA

State–action–reward–state–action (SARSA) is a method to learn a policy leveraging the TD estimation of action-value functions and an epsilon-greedy policy.

Since it is based on TD, it does not require full episodes to train, the policy can be improved at each step. We will therefore **not use the `sample_episode`** and reimplement here a similar loop.

Implement the SARSA algorithm, in this case it will be passed a policy of type `EpsilonGreedyPolicy` which stores the action-value (or Q-table). We may access and modify if using `policy.q`.

Score the trained policy.

In [ ]:
def policy_learn_sarsa(env, policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500):
    # No need to create the tables of action-values since they are stored directly in the policy
    policy.train()
    # While we haven't reached the desired number of steps
    for episode in range(n_episodes):
        # Call the policy begin_episode so it can handle epsilon decay
        policy.begin_episode(episode)
        
        # Print every couple episodes
        if not episode % print_every:
            print(f'ep: {episode}, epsilon: {policy.epsilon:.3f}')
            policy.eval()
            avg_episodes_to_goal,avg_steps_to_goal = score_policy(env, gamma, policy, 10000)
            print(f'Policy SARSA: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')
            policy.train()
            print()
        # Get initial state
        state, info = env.reset()

        # Take the first action acording to the policy
        action = policy.sample(state)

        # While we haven't reached the maximum number of steps for the episode
        for step in range(max_n_steps):     
            ### BEGIN SOLUTION
            # Perform a step of the environment
            state_next,reward,terminated,truncated,info =  env.step(action)

            # It is done if terminated or truncated
            done = terminated or truncated

            # If episode has finished
            if done:
                # Update the action-value table and leave the loop
                action_next = policy.sample(state_next)
                policy.q[state, action] += alpha * (reward + gamma * policy.q[state_next, action_next] - policy.q[state, action])
                  
                break

            # Sample the next action
            action_next = policy.sample(state_next)

            # Update the action-value table
            policy.q[state, action] += alpha * (reward + gamma * policy.q[state_next, action_next] - policy.q[state, action])
            ### END SOLUTION

            # Set the current state and action
            state = state_next
            action = action_next
            
    return

env = gym.make("FrozenLake-v1", render_mode=None)

q_initial = np.zeros((env.observation_space.n, env.action_space.n))
sarsa_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                   epsilon=1, epsilon_decay=0.999, epsilon_min=0.001)

gamma = 0.9
n_episodes = 10000
alpha = 0.2
n_episodes_score = 10000

### BEGIN SOLUTION
# Train the policy
sarsa_policy.train()
policy_learn_sarsa(env, sarsa_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

### END SOLUTION

### BEGIN SOLUTION
# Evaluate the policy
sarsa_policy.eval()
avg_episodes_to_goal,avg_steps_to_goal = score_policy(env, gamma, sarsa_policy, n_episodes_score)

### END SOLUTION

print(f'Policy SARSA: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')

In [ ]:
# ep: 0, epsilon: 1.000
# ep: 500, epsilon: 0.606
# ep: 1000, epsilon: 0.368
# ep: 1500, epsilon: 0.223
# ep: 2000, epsilon: 0.135
# ep: 2500, epsilon: 0.082
# ep: 3000, epsilon: 0.050
# ep: 3500, epsilon: 0.030
# ep: 4000, epsilon: 0.018
# ep: 4500, epsilon: 0.011
# Policy SARSA: 11.40% / 24.82

## Exercise : Q-learning

Q-learning is very similar to SARSA. **SARSA is an on-policy learning method** in which the action-values are updated following the same policy. In SARSA the action-values are updated using the value of the next state and next action taken.

Q-learning is off-policy, it does not assume the same policy when updating the action-values, instead it assumes an optimal policy by using the maximum value of the next state.

Implement a modified version of `policy_learn_sarsa` that performs Q-learning.

In [ ]:
def policy_learn_qlearn(env, policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500):
    # No need to create the tables of action-values since they are stored directly in the policy
    policy.train()
    # While we haven't reached the desired number of steps
    for episode in range(n_episodes):
        # Call the policy begin_episode so it can handle epsilon decay
        policy.begin_episode(episode)
        
        # Print every couple episodes
        if not episode % print_every:
            print(f'ep: {episode}, epsilon: {policy.epsilon:.3f}')
            policy.eval()
            avg_episodes_to_goal,mean = score_policy(env, gamma, policy, 100)
            print(f'Policy Q-learn: {avg_episodes_to_goal}, mean"{mean}')
            print()
            policy.train()
        # Get initial state
        state, info = env.reset()

        # Take the first action acording to the policy
        action = policy.sample(state)

        # While we haven't reached the maximum number of steps for the episode
        for step in range(max_n_steps):     
            ### BEGIN SOLUTION
            # Perform a step of the environment
            state_next,reward,terminated,truncated,info =  env.step(action)

            # It is done if terminated or truncated
            done = terminated or truncated


            # If episode has finished
            if done:
                # Update the action-value table and leave the loop
                policy.q[state, action] += alpha*(reward + gamma*np.max(policy.q[state_next]) - policy.q[state,action]) 
                break

            # Sample the next action
            action_next = policy.sample(state_next)

            # Update the action-value table
            policy.q[state, action] += alpha*(reward + gamma*np.max(policy.q[state_next]) - policy.q[state,action])
            ### END SOLUTION

            # Set the current state and action
            state = state_next
            action = action_next
            
    return

# env = gym.make("FrozenLake-v1", render_mode=None)

# q_initial = np.zeros((env.observation_space.n, env.action_space.n))
# qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
#                                     epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

# gamma = 0.9
# n_episodes = 5000
# alpha = 0.2
# n_episodes_score = 1000

# ### BEGIN SOLUTION
# # Train the policy
# qlearn_policy.train()
# policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

# ### END SOLUTION

# ### BEGIN SOLUTION
# # Evaluate the policy
# qlearn_policy.eval()
# avg_episodes_to_goal,avg_steps_to_goal = score_policy(env, gamma, qlearn_policy, n_episodes_score)


# ### END SOLUTION

# print(f'Policy Q-learn: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')


In [ ]:
# ep: 0, epsilon: 1.000
# ep: 500, epsilon: 0.606
# ep: 1000, epsilon: 0.368
# ep: 1500, epsilon: 0.223
# ep: 2000, epsilon: 0.135
# ep: 2500, epsilon: 0.082
# ep: 3000, epsilon: 0.050
# ep: 3500, epsilon: 0.030
# ep: 4000, epsilon: 0.018
# ep: 4500, epsilon: 0.011
# Policy Q-learn: 50.50% / 30.29

## (Optional) Exercise : Train for Taxi

Train an epsilon-greedy policy using Q-learning on the `Taxi-v3` environment.

Score the performance of this policy and compare it to a uniform policy.

In [ ]:
### BEGIN SOLUTION
env = gym.make("Taxi-v3", render_mode=None)

q_initial = np.zeros((env.observation_space.n, env.action_space.n))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 5000
alpha = 0.2
n_episodes_score = 1000

### BEGIN SOLUTION
# Train the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

### END SOLUTION

### BEGIN SOLUTION
# Evaluate the policy
qlearn_policy.eval()
avg_episodes_to_goal,avg_steps_to_goal = score_policy(env, gamma, qlearn_policy, n_episodes_score)


### END SOLUTION

print(f'Policy Q-learn: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')




### END SOLUTION

## (Optional) Exercise : Render Taxi

Run 3 episodes of a Q-learn trained policy on `Taxi-v3` this time rendering the result.

In [ ]:
### BEGIN SOLUTION
env = gym.make("Taxi-v3", render_mode=None)
env = RecordVideo(gym.make("Taxi-v3", render_mode="rgb_array"))
qlearn_policy.eval()
for i in range(5):
    states, actions, rewards, dones = sample_episode(env, qlearn_policy)
display(env.video())

### END SOLUTION

## (Bonus) Exercise : A observation space environment

Try to perform Q-learning on an environment (e.g. `CartPole-v1`) with continuous action and observation spaces.

You will need to discretize the observation space.

In [ ]:
# Example of rendering and showing a CartPole-v1 environment
env = RecordVideo(gym.make("CartPole-v1", render_mode="rgb_array"))
print(env.action_space)
print(env.observation_space)

observation, info = env.reset()
i=0
while True:
    env.render()
    
    action = env.action_space.sample() 
    state_next, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    
    if done: 
      break
            
env.close()

In [ ]:
# import numpy as np

# class ObservationWrapper(gym.ObservationWrapper):
#     def __init__(self, env, n_states=4096, bounds=[(-4.8,4.8),(-50,50),( -0.418, 0.418),(-20,20)]):
#         super().__init__(env)

#         self.n_states = n_states

#         assert isinstance(env.observation_space, gym.spaces.box.Box)

#         ### BEGIN SOLUTION
#         # Get the dimensions of the observation space
#         self.dimension = env.observation_space.shape[0]
#         self.states_per_dimention = n_states**0.25

#         self.cart_pos_bound = bounds[0]
#         self.cart_vel_bound = bounds[1]
#         self.pole_angle_bound = bounds[2]
#         self.pole_angleVel_bound = bounds[3]

        
#         self.cart_pos_step = (self.cart_pos_bound[1]- self.cart_pos_bound[0])/self.states_per_dimention
#         self.cart_vel_step = (self.cart_vel_bound[1]- self.cart_vel_bound[0])/self.states_per_dimention
#         self.pole_angle_step = (self.pole_angle_bound[1] - self.pole_angle_bound[0])/self.states_per_dimention
#         self.pole_angleVel_step = (self.pole_angleVel_bound[1]- self.pole_angleVel_bound[0])/self.states_per_dimention
        

#         ### END SOLUTION
        
#     def observation(self, obs):
#         ### BEGIN SOLUTION
#         # Quantize the observations (states)
#         new_obs = [0,0,0,0]
#         new_obs[0] = int((obs[0]-self.cart_pos_bound[0])/self.cart_pos_step)
#         new_obs[1] = int((obs[1]-self.cart_vel_bound[0])/self.cart_vel_step)
#         new_obs[2] = int((obs[2]-self.pole_angle_bound[0])/self.pole_angle_step)
#         new_obs[3] = int((obs[3]-self.pole_angleVel_bound[0])/self.pole_angleVel_step)

#         # Once quantized each dimension compute a single observation index
#         return int(new_obs[0]*self.states_per_dimention**3+new_obs[1]*self.states_per_dimention**2+new_obs[2]*self.states_per_dimention**1 + new_obs[3]*self.states_per_dimention**0)

#         ### END SOLUTION
#         # return new_obs

# # Initialize environment, wrap to render
# env = gym.make("CartPole-v1", render_mode="rgb_array")

# # Wrap to discretize the observation space
# env = ObservationWrapper(env)

# ### BEGIN SOLUTION
# # Initialize policy

# observation, info = env.reset()

# while True:
#     env.render()
    
#     action = env.action_space.sample() 
#     # print(action)
#     state_next, reward, terminated, truncated, info = env.step(action)
#     print(state_next)
#     done = terminated or truncated
#     # i+=1
#     # if i==50:
#     #     break
#     if done: 
#       break
            
# env.close()


In [ ]:
import numpy as np

class ObservationWrapper(gym.ObservationWrapper):
    '''
    n_states(int): number of state of the env.
                It has to be the some power of N = env.observation_space.shape[0]
    bounds(list of tuples): bounds of the states of the env.(taken from the gymnasium documentation).
                each tuple gives the bounds of the states
    '''
    def __init__(self, env, n_states=4096, bounds=[(-4.8,4.8),(-50,50),( -0.418, 0.418),(-20,20)]):
        super().__init__(env)

        self.n_states = n_states
        self.bounds = bounds
        assert isinstance(env.observation_space, gym.spaces.box.Box)

        # Get the dimensions of the observation space
        self.dimension = env.observation_space.shape[0] 
        self.states_per_dimention = int(n_states**(1/self.dimension)) #we give equal number of state per dimension
        
        # calculating steps for each state and storing in a list
        self.step_list = []
        for state_bound in bounds:
            step = (state_bound[1] - state_bound[0])/self.states_per_dimention
            self.step_list.append(step)

        
    def get_quant_observation(self, new_obs):
        ##calculate the new quantized state index from a list of states.  (like converting number systems)
        # print(new_obs)
        quantized_obs = 0
        dimension = self.dimension
        for i, obs in enumerate(new_obs):
            quantized_obs+= obs*self.states_per_dimention**(dimension-i-1)
        
        # print(quantized_obs)
        return int(quantized_obs)

    def observation(self, obs):
        ### BEGIN SOLUTION
        # Quantize the observations (states)
        new_obs = []
        # print(obs)
        for i,observation_state in enumerate(obs):
            new_obs.append(int((observation_state-self.bounds[i][0])/self.step_list[i]))
            
        # Once quantized each dimension compute a single observation index
        return self.get_quant_observation(new_obs)

        


In [ ]:
def score_policy(env, gamma, policy, n_episodes):
    episodes_to_goal = 0
    steps_to_goal = []

    ### BEGIN SOLUTION
    for episode in range(n_episodes):
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        episodes_to_goal+= len(states)




    ### END SOLUTION

    return episodes_to_goal/n_episodes,steps_to_goal

In [ ]:
# Initialize environment, wrap to render
env = gym.make("CartPole-v1", render_mode="rgb_array")

# Wrap to discretize the observation space
env = ObservationWrapper(env)

### BEGIN SOLUTION
# Initialize policy

observation, info = env.reset()

q_initial = np.zeros((4096, 2))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 20000
alpha = 0.1
n_episodes_score = 100

### BEGIN SOLUTION
# Train the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

### END SOLUTION

### BEGIN SOLUTION
# Evaluate the policy
qlearn_policy.eval()
avg_episodes_to_goal = score_policy(env, gamma, qlearn_policy, n_episodes_score)


### END SOLUTION

print(f'Policy Q-learn: {avg_episodes_to_goal}')



In [ ]:
qlearn_policy.eval()
avg_episodes_to_goal = score_policy(env, gamma, qlearn_policy, n_episodes_score)


### END SOLUTION

print(f'Policy Q-learn: {avg_episodes_to_goal}')


In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")

# Wrap to discretize the observation space
# env = ObservationWrapper(env)

env = RecordVideo(ObservationWrapper(env))
qlearn_policy.eval()
for i in range(1):
    state_next, reward, terminated, truncated = sample_episode(env, qlearn_policy)
    
env.close()

In [ ]:
env = RecordVideo(gym.make("Acrobot-v1", render_mode="rgb_array"))
print(env.action_space)
print(env.observation_space)

observation, info = env.reset()
i=0
while True:
    env.render()
    
    action = env.action_space.sample() 
    state_next, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    
    if done: 
      break
            
env.close()

In [ ]:
def score_policy(env, gamma, policy, n_episodes):
    episodes_to_goal = 0
    steps_to_goal = []

    ### BEGIN SOLUTION
    for episode in range(n_episodes):
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        if rewards[-1]==0:
            episodes_to_goal+=1
            steps_to_goal.append(len(rewards))




    ### END SOLUTION

    return episodes_to_goal/n_episodes, np.mean(steps_to_goal)




    ### END SOLUTION


In [ ]:
from inspect import BoundArguments
env = gym.make("Acrobot-v1", render_mode="rgb_array")

# Wrap to discretize the observation space
n_states = env.observation_space.shape[0]**5
action_space = env.action_space.n
bounds = [(-1,1),(-1,1),(-1,1),(-1,1),(-12.567,12.567),(-28.274, 28.274 )]
env = ObservationWrapper(env, n_states = n_states, bounds = bounds)

### BEGIN SOLUTION
# Initialize policy

observation, info = env.reset()

q_initial = np.zeros((n_states, action_space))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.999, epsilon_min=0.001)

gamma = 0.9
n_episodes = 5000
alpha = 0.2
n_episodes_score = 100

### BEGIN SOLUTION
# Train the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

### END SOLUTION

### BEGIN SOLUTION
# Evaluate the policy
qlearn_policy.eval()
avg_episodes_to_goal, mean_steps = score_policy(env, gamma, qlearn_policy, n_episodes_score)


### END SOLUTION

print(f'Policy Q-learn: {avg_episodes_to_goal:.3%}, avg steps: {mean_steps}')



In [ ]:
env = gym.make("Acrobot-v1", render_mode="rgb_array")

# Wrap to discretize the observation space
n_states = env.observation_space.shape[0]**5
action_space = env.action_space.n
bounds = [(-1,1),(-1,1),(-1,1),(-1,1),(-12.567,12.567),(-28.274, 28.274 )]
env = ObservationWrapper(env, n_states = n_states, bounds = bounds)
env = RecordVideo(env)

qlearn_policy.eval()
for i in range(2):
    state_next, reward, terminated, truncated = sample_episode(env, qlearn_policy)
    
env.close()

In [ ]:
pip install gymnasium[box2d]

In [ ]:
env = RecordVideo(gym.make(
    "LunarLander-v2",
    continuous = False,
    gravity = -10.0,
    enable_wind = False,
    #wind_power = 15.0,
    #turbulence_power = 1.5,
    render_mode="rgb_array"
))
observation, info = env.reset()
i=0
while True:
    env.render()
    
    action = env.action_space.sample() 
    state_next, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    # print(reward)
    if done: 
      break
            
env.close()

In [ ]:
def score_policy(env, gamma, policy, n_episodes):
    episodes_to_goal = []
    steps_to_goal = []

    ### BEGIN SOLUTION
    for episode in range(n_episodes):
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        # if rewards[-1]==100:
        #     episodes_to_goal+=1
        #     steps_to_goal.append(len(rewards))
        episodes_to_goal.append(rewards[-1])
        steps_to_goal.append(len(actions))


    ### END SOLUTION

    return np.mean(episodes_to_goal), np.mean(steps_to_goal)

env = gym.make(
    "LunarLander-v2",
    continuous = False,
    gravity = -10.0,
    enable_wind = False,
    #wind_power = 15.0,
    #turbulence_power = 1.5,
    render_mode="rgb_array"
)

# Wrap to discretize the observation space
n_states = env.observation_space.shape[0]**6
action_space = env.action_space.n

bounds = [(-1.5,1.5),(-1.5,1.5), (-5,5),(-5,5),(-3.14,3.14),(-5, 5 ),(-0,1),(-0,1)]
env = ObservationWrapper(env, n_states = n_states, bounds = bounds)

### BEGIN SOLUTION
# Initialize policy

observation, info = env.reset()

q_initial = np.zeros((n_states, action_space))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.9999999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 500000
alpha = 0.2
n_episodes_score = 100

### BEGIN SOLUTION
# Train the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

### END SOLUTION

### BEGIN SOLUTION
# Evaluate the policy
qlearn_policy.eval()
avg_episodes_to_goal, mean_steps = score_policy(env, gamma, qlearn_policy, n_episodes_score)


### END SOLUTION

print(f'Policy Q-learn: {avg_episodes_to_goal}, avg steps: {mean_steps}')



env = gym.make(
    "LunarLander-v2",
    continuous = False,
    gravity = -10.0,
    enable_wind = False,
    #wind_power = 15.0,
    #turbulence_power = 1.5,
    render_mode="rgb_array"
)

# Wrap to discretize the observation space
n_states = env.observation_space.shape[0]**6
action_space = env.action_space.n

bounds = [(-1.5,1.5),(-1.5,1.5), (-5,5),(-5,5),(-3.14,3.14),(-5, 5 ),(-0,1),(-0,1)]
env = ObservationWrapper(env, n_states = n_states, bounds = bounds)
env = RecordVideo(env)

qlearn_policy.eval()
for i in range(2):
    state_next, reward, terminated, truncated = sample_episode(env, qlearn_policy)
    
env.close()

ep: 0, epsilon: 1.000
Policy Q-learn: -100.0, mean"69.74

ep: 500, epsilon: 1.000
Policy Q-learn: -100.0, mean"197.26

ep: 1000, epsilon: 1.000
Policy Q-learn: -99.00114372062893, mean"188.78

ep: 1500, epsilon: 1.000
Policy Q-learn: -100.0, mean"142.22

ep: 2000, epsilon: 1.000
Policy Q-learn: -100.0, mean"190.13

ep: 2500, epsilon: 1.000
Policy Q-learn: -100.0, mean"219.95

ep: 3000, epsilon: 1.000
Policy Q-learn: -100.0, mean"161.39

ep: 3500, epsilon: 1.000
Policy Q-learn: -99.0225654842259, mean"343.7

ep: 4000, epsilon: 1.000
Policy Q-learn: -100.0, mean"309.67

ep: 4500, epsilon: 1.000
Policy Q-learn: -100.0, mean"155.57

ep: 5000, epsilon: 1.000
Policy Q-learn: -96.99569330648364, mean"158.17

ep: 5500, epsilon: 0.999
Policy Q-learn: -96.00129699156678, mean"158.3

ep: 6000, epsilon: 0.999
Policy Q-learn: -100.0, mean"308.13

ep: 6500, epsilon: 0.999
Policy Q-learn: -98.00410827575901, mean"227.07

ep: 7000, epsilon: 0.999
Policy Q-learn: -100.0, mean"312.55

ep: 7500, epsilon:

In [ ]:
#action space wrapper

class ObservationWrapper(gym.ObservationWrapper):
    '''
    n_states(int): number of state of the env.
                It has to be the some power of N = env.observation_space.shape[0]
    bounds(list of tuples): bounds of the actions of the env.(taken from the gymnasium documentation).
                each tuple gives the bounds of the states
    '''
    def __init__(self, env, n_states=4096, bounds=[(-4.8,4.8),(-50,50),( -0.418, 0.418),(-20,20)]):
        super().__init__(env)

        self.n_states = n_states
        self.bounds = bounds
        assert isinstance(env.observation_space, gym.spaces.box.Box)

        # Get the dimensions of the observation space
        self.dimension = env.observation_space.shape[0] 
        self.states_per_dimention = int(n_states**(1/self.dimension)) #we give equal number of state per dimension
        
        # calculating steps for each state and storing in a list
        self.step_list = []
        for state_bound in bounds:
            step = (state_bound[1] - state_bound[0])/self.states_per_dimention
            self.step_list.append(step)

        
    def get_quant_observation(self, new_obs):
        ##calculate the new quantized state index from a list of states.  (like converting number systems)
        # print(new_obs)
        quantized_obs = 0
        dimension = self.dimension
        for i, obs in enumerate(new_obs):
            quantized_obs+= obs*self.states_per_dimention**(dimension-i-1)
        
        # print(quantized_obs)
        return int(quantized_obs)

    def observation(self, obs):
        ### BEGIN SOLUTION
        # Quantize the observations (states)
        new_obs = []
        # print(obs)
        for i,observation_state in enumerate(obs):
            new_obs.append(int((observation_state-self.bounds[i][0])/self.step_list[i]))
            
        # Once quantized each dimension compute a single observation index
        return self.get_quant_observation(new_obs)
